<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/03%20Machine%20Learning/notebooks/05-SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVM

En esta notebook mostraremos el uso del clasificador **SVM** (Support Vector Machine). Realizaremos un ejemplo con datos artificiales, con fines didácticos, y un ejemplo más grande, con datos reales.

Usaremos la implementación de sklearn, llamada [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) (Support Vector Classifier)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#@title Funciones que necesitamos para graficar las fronteras de decisión

def make_meshgrid(x, y, h=.02):
    '''
    función para hacer la malla de puntos para colorear las regiones de decisión,
    la malla de puntos abarca la región donde se encuentran los puntos (x,y)
    'h' es el tamaño de paso
    '''
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

## Ejemplo 1: Un caso linealmente separable

En este primer ejemplo mostramos el uso básico del SVM como clasificador en un ejemplo ilustrativo.

### El conjunto de datos

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=200,
                  n_features=2,
                  centers=2,
                  random_state=57)

print(X.shape, y.shape)

plt.figure()
plt.scatter(X[:,0], X[:,1],
            c=y # Observar que el color lo especificamos con el arreglo de etiquetas
            )
plt.show()

### La división *train/test*

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2023)

print(f"X Train: {X_train.shape}")
print(f"X Test: {X_test.shape}")
print(f"Y Train: {y_train.shape}")
print(f"Y Test: {y_test.shape}")

### Entrenamiento

Entrenemos y evaluemos el clasificador en el conjunto de entrenamiento

In [ ]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
clf.score(X_train,y_train)  # Este score es el accuracy

### Evaluación

Evaluemos varias métricas en el conjunto de prueba

In [ ]:
clf.score(X_test,y_test)    # Este score es el accuracy

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, confusion_matrix

y_pred = clf.predict(X_test)

cm = confusion_matrix(y_test,y_pred)

plt.figure()
plt.imshow(cm)
plt.colorbar()
plt.xticks([0,1])
plt.yticks([0,1])
plt.xlabel("Etiqueta predicha")
plt.ylabel("Etiqueta real")
plt.show()

In [ ]:
print(f"Accuracy: {accuracy_score(y_test,y_pred)}")
print(f"Recall: {recall_score(y_test,y_pred)}")
print(f"Precision: {precision_score(y_test,y_pred)}")
print(f"F1 Score: {f1_score(y_test,y_pred)}")

Obtuvimos métricas de rendimiento del 100%, **¿te parece sospechoso?**

In [ ]:
xx, yy = make_meshgrid(X[:,0], X[:,1]) # Hacemos el grid para graficar las regiones

fig, (ax1, ax2) = plt.subplots(1,2,dpi=100,figsize=(10,4)) # El parámetro dpi especifíca los puntos por pulgada (DPI) de la imagen

fig.suptitle("Fronteras de decisión")

plot_contours(ax1, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax1.scatter(X_train[:,0], X_train[:,1], c=y_train, cmap=plt.cm.coolwarm, s=20)
ax1.set_xticks(())
ax1.set_yticks(())
ax1.set_title('Conjunto de entrenamiento')

plot_contours(ax2, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax2.scatter(X_test[:,0], X_test[:,1], c=y_test, cmap=plt.cm.coolwarm, s=20)
ax2.set_xticks(())
ax2.set_yticks(())
ax2.set_title('Conjunto de prueba')

plt.show()

Algunos hiperparámetros:

* `C`:  Este parámetro es fundamental para encontrar un equilibrio adecuado entre el ajuste del modelo a los datos de entrenamiento y la capacidad del modelo para generalizar a datos nuevos.
* `kernel`: Tipo de kernel usado.


Algunos métodos:

*   `score()`: Accuracy del clasificador en el conjunto especificado.
*   `predict()`: Predecir la clase del conjunto especificado usando el modelo ya entrenado.



## Ejemplo 2: Un caso no linealmente separable

### El conjunto de datos

Creamos un conjunto de datos con una condición XOR

In [ ]:
np.random.seed(17) # Fijamos un seed para la reproducibilidad de resultados

X = np.random.randn(1000, 2)
Y = np.array([int(np.logical_xor(x[0] > 0, x[1] > 0)) for x in X])

plt.figure()
plt.scatter(X[Y==0, 0], X[Y==0, 1], s=20, color='blue', label='Clase 0')
plt.scatter(X[Y==1, 0], X[Y==1, 1], s=20, color='red',label='Clase 1')
plt.legend()
plt.show()

### Separación *train/test*

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2023)

print(f"X Train: {x_train.shape}")
print(f"X Test: {x_test.shape}")
print(f"Y Train: {y_train.shape}")
print(f"Y Test: {y_test.shape}")

### Clasificación

#### SVM lineal

Entrenemos el clasificador usando el kernel lineal. Observar que, por default, $C=1$.

In [ ]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(x_train, y_train)

Observemos el accuracy en el conjunto de entrenamiento y prueba. En este caso, el método `score` de la clase `SVC` calcula el accuracy.

In [ ]:
print(f"Training mean accuracy: {round(clf.score(x_train, y_train),3)}")
print(f"Test mean accuracy: {round(clf.score(x_test, y_test),3)}")

Observemos la frontera de decisión calculada por el clasificador y los conjuntos de entrenamiento y prueba.

In [ ]:
xx, yy = make_meshgrid(X[:,0], X[:,1]) # Hacemos el grid para graficar las regiones

fig, (ax1, ax2) = plt.subplots(1,2,dpi=100,figsize=(10,4)) # El parámetro dpi especifíca los puntos por pulgada (DPI) de la imagen

fig.suptitle("Fronteras de decisión")

plot_contours(ax1, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax1.scatter(x_train[:,0], x_train[:,1], c=y_train, cmap=plt.cm.coolwarm, s=20)
ax1.set_xticks(())
ax1.set_yticks(())
ax1.set_title('Conjunto de entrenamiento')

plot_contours(ax2, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax2.scatter(x_test[:,0], x_test[:,1], c=y_test, cmap=plt.cm.coolwarm, s=20)
ax2.set_xticks(())
ax2.set_yticks(())
ax2.set_title('Conjunto de prueba')

plt.show()

#### ⭕ Probar otros kernels

Con los mismos conjuntos de prueba y entrenamiento:

1. Repetir el experimento de clasificación de arriba, usando otros kernels.
2. En cada caso que pruebes grafica los puntos (los de prueba) y la frontera de decisión.
3. En cada caso, reporta el valor de accuracy y recall, usando el conjunto de prueba solamente.

**¿Qué kernel parece dar mejor resultado?**

Documentación: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

* El kernel lineal es mejor para datos linealmente separables. Es una opción cuando el conjunto de datos es grande.
* El kernel Gaussiano (RBF) tiende a dar buenos resultados cuando no se tiene información adicional sobre los datos.
* Los kernels polinomiales tienden a dar buenos resultados cuando los datos de entrenamiento están normalizados.

[El truco del kernel](https://www.geogebra.org/m/xawkavxe)

⭕ Prueba también con otros valores de `C` y repite los pasos de arriba, ¿qué efecto tiene el modificar este valor en la clasificación?

### Usando gridsearch para encontrar los mejores parámetros

[GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) toma un estimador (por ejemplo, SVM) y un conjunto de parámetros del estimador. Sobre estos parámetros hace una busqueda para encontrar la combinación de parámetros que da mejores resultados en el estimador.

GridSearchCV tiene métodos “fit” y “score” method, entre otros. Es decir, no es necesario tomar los parámetros e introducirlos en el estimador.

In [ ]:
from sklearn.model_selection import GridSearchCV

Encuentra los mejores parámetros para el clasificador SVM utilizando grid search. Guíate por el desempeño en el set de entrenamiento y validación.

Prueba los siguientes hyperparámetros.
* kernel = linear, polynomial, rbf
* C = 0.01, 0.1, 1.0, 10, 100
* grado del polinomio = 1, 2, 3, 4 (solo para el kernel polinomial)
* gamma = auto, scale:

Definimos los parámetros sobre los que se hará la busqueda

In [ ]:
param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'kernel': ('linear', 'poly', 'rbf'),
              'degree': [1, 2, 3, 4], 'gamma': ('auto', 'scale')}
param_grid

Realizamos una busqueda sobre estos parámetros

In [ ]:
clf = SVC()
gs = GridSearchCV(clf, param_grid)
gs.fit(x_train, y_train)

Veamos los mejores hiper-parámetros

In [ ]:
print(f"Best score: {gs.best_score_:.4f}")
print(f"Best params: {gs.best_params_}")

Definamos un clasificador SVM con estos mejores hiperparámetros

In [ ]:
best_svm = SVC(C=100, kernel='poly', degree=2, gamma='auto')
best_svm.fit(x_train, y_train)

print(f"Train mean accuracy: {best_svm.score(x_train, y_train):6.4f}")
print(f"Test mean accuracy: {best_svm.score(x_test, y_test):6.4f}")

Graficamos la frontera de decisión

In [ ]:
xx, yy = make_meshgrid(X[:,0], X[:,1]) # Hacemos el grid para graficar las regiones

fig, ax = plt.subplots(dpi=100)  # El parámetro dpi especifíca los puntos por pulgada (DPI) de la imagen
plot_contours(ax, best_svm, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax.scatter(X[:,0], X[:,1], c=Y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
ax.set_xticks(())
ax.set_yticks(())
ax.set_title('Frontera de decisión del SVM')
plt.show()

## Ejemplo 3: MNIST

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()

X = digits.data
y = digits.target

print(X.shape, y.shape)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.subplot(1,2,1)
plt.imshow(X[0].reshape(8,8))
plt.title(y[0])
plt.subplot(1,2,2)
plt.imshow(X[1].reshape(8,8))
plt.title(y[1])
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=99,train_size=0.8)

print(X_train.shape)
print(X_test.shape)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC()
clf.fit(X_train,y_train)
y_train_pred = clf.predict(X_train)
y_pred = clf.predict(X_test)
print(f"Train Accuracy: {accuracy_score(y_train,y_train_pred)}")
print(f"Test Accuracy: {accuracy_score(y_test,y_pred)}")

## Ejemplo 4: Un dataset grande y muy desvalanceado

Para este problema usaremos el datset de Kaggle.

**Contexto**

Los conjuntos de datos contienen transacciones realizadas con tarjetas de crédito en septiembre de 2013 por titulares de tarjetas europeos. Este conjunto de datos presenta transacciones que ocurrieron en dos días, donde tenemos 492 fraudes de 284,807 transacciones. El conjunto de datos está altamente desequilibrado, la clase positiva (fraudes) representa el 0.172% de todas las transacciones.

Contiene solo variables de entrada numéricas que son el resultado de una transformación PCA. Desafortunadamente, debido a problemas de confidencialidad, no se pueden obtener las características originales y más información de fondo sobre los datos. Las características $V_1$, $V_2$, ..., $V_{28}$ son los componentes principales obtenidos con PCA, las únicas características que no se han transformado con PCA son 'Tiempo' y 'Cantidad'. La función 'Tiempo' contiene los segundos transcurridos entre cada transacción y la primera transacción en el conjunto de datos. La característica 'Cantidad' es la Cantidad de la transacción, esta característica se puede utilizar para el aprendizaje sensible al costo dependiente del ejemplo. La característica 'Clase' es la variable de respuesta y toma el valor 1 en caso de fraude y 0 en caso contrario.


Recordemos las buenas prácticas del Machine Learning: https://scikit-learn.org/stable/common_pitfalls.html

In [ ]:
!wget https://github.com/DCDPUAEM/DCDP/raw/main/03%20Machine%20Learning/data/creditcard.zip

In [ ]:
!unzip creditcard.zip

In [ ]:
import pandas as pd

credito = pd.read_csv("creditcard.csv")

In [ ]:
credito.head()

In [ ]:
credito.describe()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()

# Graficamos los que no son fraude
time_amount = credito[credito['Class'] == 0][['Time','Amount']].values
plt.scatter(time_amount[:,0], time_amount[:,1],
            c='green',alpha=0.25,label='Clean')
# Graficamos los que sí son fraude
time_amount = credito[credito['Class'] == 1][['Time','Amount']].values
plt.scatter(time_amount[:,0], time_amount[:,1],
            c='red',label='Fraud',marker='x')
plt.legend(loc='best')
plt.title('Amount vs fraud')
plt.xlabel('Time', fontsize=16)
plt.ylabel('Amount', fontsize=16)
plt.show()

In [ ]:
import seaborn as sns

plt.figure()
sns.countplot(x = "Class", data = credito)
plt.show()

In [ ]:
import numpy as np

No_of_frauds = credito[credito["Class"]==1].shape[0]
No_of_normals = credito[credito["Class"]==0].shape[0]
print("Hay {} transacciones normales (clase 0)".format(No_of_normals))
print("Hay {} transacciones fraudulentas (clase 1)".format(No_of_frauds))
total = No_of_frauds + No_of_normals
pf= (No_of_frauds / total)*100
pn= (No_of_normals / total)*100
print("Porcentaje clase 0 = {}%".format(np.round(pn,2)))
print("Porcentaje clase 1 = {}%".format(np.round(pf,2)))

### Submuestro

Se necesita hacer un submuestreo para balancear las clases

* Está claro que la Clase 1 está subrepresentada ya que solo  representa el 0.17% de todo el conjunto de datos.
* Si entrenamos nuestro modelo usando este conjunto de datos, el modelo será ineficiente y será entrenado para predecir solo la Clase 0 porque no tendrá suficientes datos de entrenamiento.
* Podemos obtener una alta exactitud al probar el modelo, pero no debemos confundirnos con esto porque nuestro conjunto de datos no tiene datos de prueba equilibrados. Por lo tanto, tenemos que confiar en el recall que se basa en TP y FP.
* En los casos en que tengamos datos asimétricos, agregar datos adicionales de la característica subrepresentada (sobremuestreo) es una opción, mediante la modelación de la distribución de los datos. Por ahora no tenemos esa opción, así que tendremos que recurrir al submuestreo.
* El submuestreo del conjunto de datos implica mantener todos nuestros datos subrepresentados (Clase 1) mientras se muestrea el mismo número de características de la Clase 0 para crear un nuevo conjunto de datos que comprenda una representación igual de ambas clases.

Obtenemos un conjunto de datos más balanceado que contenga el doble de instancias no fraudulentas respecto a las fraudulentas

In [ ]:
# lista los índices de fraude del data set completo
fraud_idxs = credito[credito["Class"]==1].index.to_list()

# lista de índices normales del data set completo
normal_idxs = credito[credito["Class"]==0].index.to_list()

# seleccionamos aleatoriamente el doble de índices de transacciones normales que de normales
random_normal_idxs = np.random.choice(normal_idxs, No_of_frauds*2, replace= False)

# concatenamos los índices fraudulentos y normales y creamos el dataframe sub-sampleado
undersampled_indices = np.concatenate([fraud_idxs, random_normal_idxs])
undersampled_data = credito.iloc[undersampled_indices, :]

print(f"Fraude: {len(fraud_idxs)}, Normales: {len(random_normal_idxs)}")
undersampled_data.head()

Comprobemos que los datos quedaron balanceados

In [ ]:
No_of_frauds_sampled = len(undersampled_data[undersampled_data["Class"]== 1])

No_of_normals_sampled = len(undersampled_data[undersampled_data["Class"]== 0])

print("Número de transacciones normales (clase 0): ", No_of_normals_sampled)
print("Número de transacciones fraudulentas (clase 1): ", No_of_frauds_sampled)
total_sampled = No_of_frauds_sampled + No_of_normals_sampled
print("Número total de instancias: ", total_sampled)

Fraud_percent_sampled = (No_of_frauds_sampled / total_sampled)*100
Normal_percent_sampled = (No_of_normals_sampled / total_sampled)*100
print(f"Porcentaje clase 0: {round(Normal_percent_sampled,2)}")
print(f"Porcentaje clase 1: {round(Normal_percent_sampled,2)}")

count_sampled = pd.value_counts(undersampled_data["Class"], sort= True)
count_sampled.plot(kind= 'bar')
plt.show()

In [ ]:
# quitamos las columnas "Time" y "Amount"
undersampled_data.drop(["Time"], axis= 1,inplace=True)

### Oversampling

Ahora haremos un proceso llamado [SMOTE: Synthetic Minority Over-sampling Technique](https://arxiv.org/abs/1106.1813)


Para ello necesitamos instalar la librería de _aprendizaje desequilibrado_ ``imbalanced-learn`` de Python

In [ ]:
!pip install imbalanced-learn

Podemos imprimir información sobre el módulo

In [ ]:
import imblearn
print(imblearn.__version__)

Obtenemos la matriz de datos $X$ y el vector de clases $y$ correspondiente

In [ ]:
X = undersampled_data.loc[:, undersampled_data.columns != "Class"].values
y = undersampled_data.loc[:, undersampled_data.columns == "Class"].values

print(f"Matriz de features: {X.shape}")
print(f"Matriz de etiquetas: {y.shape}")

Hagamos el proceso de sobre-muestreo [SMOTE](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html)

In [ ]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
X_oversampled, y_oversampled = oversample.fit_resample(X, y)

Verifiquemos la cantidad de datos ahora

In [ ]:
from collections import Counter

print(f"Matriz de features: {X_oversampled.shape}")
print(f"Matriz de etiquetas: {y_oversampled.shape}")

print(Counter(y_oversampled))

### Crear el conjunto de entrenamiento y prueba

Separamos los datos en datos de entrenamiento (75%) y prueba (25%)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X_oversampled, y_oversampled,
                                                    test_size = 0.25,
                                                    random_state = 359)

print(f"X_train: {len(X_train)}")
print(f"X_test: {len(X_test)}")
print(f"y_train: {len(y_train)}")
print(f"y_test: {len(y_test)}")

### Re-escalemos los datos

In [ ]:
from sklearn import preprocessing

sc = preprocessing.StandardScaler()
X_train = sc.fit_transform(X_train)

X_test = sc.transform(X_test)

⭕ Elige una SVM y entrénalo con un conjunto de parámetros de tu elección. Obtener el accuracy usando el método `score` del clasificador.

In [ ]:
classifier = SVC(C=1, kernel= 'rbf', random_state=0, gamma='scale')
classifier.fit(X_train, y_train)
classifier.score(X_train, y_train)

### Prueba el modelo

Realiza las predicciones con el conjunto de prueba y bserva la matriz de confusión.

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = classifier.predict(X_test)
CM = confusion_matrix(y_test, y_pred)
print(CM)

⭕ Calcula también el *F1-score* y el *precision score*

### Aplica GridSearch para obtener los mejores parámetros para una SVM

In [ ]:
parameters = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
              {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}
              ]

grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           n_jobs = -1)

grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
print("The best accuracy using gridSearch is", best_accuracy)

best_parameters = grid_search.best_params_
print("The best parameters for using this model is", best_parameters)

### Utiliza los mejores parámetros para probar de nuevo tu modelo

In [ ]:
classifier_with_best_parameters =  SVC(C= best_parameters["C"],
                                       kernel= best_parameters["kernel"],
                                       random_state= 0)
classifier_with_best_parameters.fit(X_train, y_train)

y_pred_best_parameters = classifier_with_best_parameters.predict(X_test)

CM2 = confusion_matrix(y_test, y_pred_best_parameters)
print(CM2)

⭕ Calcula las métricas de rendimiento: Accuracy, Recall, F1-score, Precision



---



### ⭕ Práctica

La práctica consiste en dos ejercicios:

1. Usa el modelo anterior (**no lo vuelvas a entrenar**) para obtener las predicciones en todos el conjunto de datos original. Reporta las 4 métricas de rendimiento, así como la matriz de confusión. *Hint*: Puedes usar un pipeline para facilitar el proceso.

2. Usa el clasificador lineal OLS con el conjunto de datos entrenamiento balanceado usado en la sesión (el de tamaño 1476). Reporta las 4 métricas de rendimiento, así como la matriz de confusión.

3. Entrena un nuevo clasificador SVM en todo el conjunto sesgado.

    3.1. Separa el conjunto completo en 75% de entrenamiento y 20% de prueba.

    3.2. Entrena un nuevo modelo en este nuevo conjunto de entrenamiento y obten las predicciones en el conjunto de prueba.
    
    3.3 Reporta las 4 métricas de rendimiento, así como la matriz de confusión.
    
    Puedes usar técnicas de re-escalamiento, gridsearch, selección de features. Puedes usar un pipeline para facilitar el proceso.

Redacta una conclusión comparando el desempeño de la parte 1 y la parte 3.



___